In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\John\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install spacy==2.2.0

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\John\anaconda3\envs\hackMT\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
#import data
df_id = pd.read_csv("dataset/nonprofit.txt", sep = "|", encoding = "cp1252")
df_text = pd.read_csv("dataset/nonprofit_text.txt", sep = "|", encoding = "cp1252")

In [8]:
#merge sets
df_merge = pd.merge(df_id, df_text, how='inner', on='nonprofit_id')

In [9]:
#filter by TN entries
df_text = df_merge.loc[(df_merge["stabbrv"]=='TN')]

In [10]:
# removes the empty descriptions
df_text = df_text[df_text["description"].str.len() > 0]

# combines all by nonprofit_id to a single description
df_text = df_text.groupby(['nonprofit_id'], as_index = False).agg({'description': ' '.join})

In [11]:
df_text = df_text[df_text["description"].str.len() > 10]

In [12]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10096 entries, 0 to 10159
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nonprofit_id  10096 non-null  int64 
 1   description   10096 non-null  object
dtypes: int64(1), object(1)
memory usage: 236.6+ KB


In [13]:
#create sample set
#df_text = df_text.sample(n = 250000)
#df_text.head()

In [14]:
data = df_text.description.values.tolist()

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1])

[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']]


In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']


In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
#python3 -m spacy download en

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

print(data_words_nostops[:1])

[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'youth', 'community', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']]


In [20]:
print(data_words_nostops[7])

['jackson', 'area', 'chamber', 'commerce', 'foundation', 'inc']


In [21]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

print(data_words_bigrams[:1])

In [22]:
print(data_words_bigrams[7])

In [23]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])


[['provide', 'sound', 'supervise', 'provide', 'sound', 'supervise', 'recreational', 'sport', 'program', 'youth', 'community', 'receive', 'sport', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program']]


In [24]:
print(data_lemmatized[7])

[]


In [25]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 1)], [(2, 2), (6, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [26]:
# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(workers = 3, corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)

In [27]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"school" + 0.070*"student" + 0.043*"education" + 0.034*"provide" + '
  '0.027*"high" + 0.019*"support" + 0.019*"college" + 0.019*"educational" + '
  '0.014*"academic" + 0.014*"scholarship"'),
 (1,
  '0.042*"program" + 0.027*"community" + 0.021*"life" + 0.012*"art" + '
  '0.010*"help" + 0.010*"skill" + 0.010*"work" + 0.010*"people" + '
  '0.010*"opportunity" + 0.009*"family"'),
 (2,
  '0.071*"provide" + 0.069*"housing" + 0.055*"income" + 0.054*"low" + '
  '0.039*"service" + 0.021*"elderly" + 0.021*"member" + 0.019*"affordable" + '
  '0.016*"union" + 0.015*"loan"'),
 (3,
  '0.073*"service" + 0.070*"provide" + 0.046*"care" + 0.041*"health" + '
  '0.022*"medical" + 0.019*"patient" + 0.018*"individual" + 0.017*"community" '
  '+ 0.016*"support" + 0.015*"assistance"'),
 (4,
  '0.030*"promote" + 0.021*"member" + 0.019*"business" + 0.017*"public" + '
  '0.015*"development" + 0.014*"state" + 0.013*"industry" + 0.012*"economic" + '
  '0.012*"organization" + 0.012*"research"'),
 (5,

In [28]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.739232197002175

Coherence Score:  0.41184461057357863


In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\John\anaconda3\envs\hackMT\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.185123  0.090373       1        1  17.484173
4      0.107843 -0.169311       2        1  13.626072
3     -0.089720  0.171277       3        1  12.582677
7     -0.054143 -0.078776       4        1   8.987502
0      0.047325 -0.063157       5        1   8.288476
6      0.091438 -0.065758       6        1   7.453428
5      0.213296  0.025828       7        1   7.030259
10    -0.037327  0.216432       8        1   6.051594
2     -0.282379  0.002627       9        1   5.770052
9      0.058005 -0.109689      10        1   4.749941
11    -0.047502  0.190348      11        1   4.627345
8     -0.191957 -0.210194      12        1   3.348483, topic_info=             Term         Freq        Total Category  logprob  loglift
278         child  2922.000000  2922.000000  Default  30.0000  30.0000
115       service  4868.000000  4868.000000  Default  29.0000  29.0000
316        school  2435.000000  2435.000000  Default  28.0000  28.0000
413       student  2080.000000  2080.000000  Default  27.0000  27.0000
6         provide  9582.000000  9582.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
275       benefit   126.396584  1001.751002  Topic12  -4.5477   1.3266
82     membership    96.402858   338.057934  Topic12  -4.8186   2.1420
227  organization   112.527516  3376.228488  Topic12  -4.6640  -0.0047
87        promote   107.088722  2215.858242  Topic12  -4.7135   0.3669
767       welfare    92.004799   333.194390  Topic12  -4.8653   2.1098

[761 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
459       8  0.977673  abandon
1240      8  0.143940    abuse
1240     11  0.140878    abuse
1240     12  0.713576    abuse
4053      9  0.991195   abused
...     ...       ...      ...
266      11  0.046225     year
3858      7  0.980191     ymca
14        1  0.456339    youth
14        6  0.424624    youth
14       11  0.118930    youth

[1482 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 8, 1, 7, 6, 11, 3, 10, 12, 9])

In [30]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
        return lda_model.id2word[word_topics[0][0]]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_model_result"] = get_readable_topic(doc)
    index = index + 1

In [31]:
df_text.head(10)

,nonprofit_id,description,topic_model_result
0,602.0,"Provide sound, supervised PROVIDE SOUND, SUPER...",baseball
1,640.0,"PROVIDE FOOD, CLOTHNG AND FURNITURE FOR ECONOM...",community
3,710.0,MCU MEDICAL FOUNDATION INC Provide financial a...,community
4,778.0,TO PROMOTE EDUCATIONAL OPPORTUNITIES FOR WOMEN...,provide
5,1071.0,PROVIDE A SUPPORTIVE LIVING SERVICES HOME AND ...,youth
6,1134.0,RELIGIOUS YOUTH CAMP GROUP RETREATS AND SUMMER...,community
7,1567.0,TO PROVIDE ASSISTANCE TO CANCER PATIENTS IN NE...,None
8,1593.0,JACKSON AREA CHAMBER OF COMMERCE FOUNDATION INC,None
9,1667.0,WILSON COUNTY BLACK HISTORY COMMITTEE,provide
10,1739.0,PROMOTE AND ENCOURAGE THE SHOWING OF REINING H...,community


In [32]:
df_text.to_csv('output.csv')